# Importing the Libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# missing value treatment
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer

# normal or not normal distribution
from scipy.stats import kstest

# scaling
from sklearn.preprocessing import StandardScaler

# country to continent (as numeric classes)
import pycountry_convert as pc

# importing metrics
from sklearn.metrics import r2_score

# vif check
from statsmodels.stats.outliers_influence import variance_inflation_factor

# train-test split
from sklearn.model_selection import train_test_split

# hyperparameter tuning
from sklearn.model_selection import cross_val_score
import optuna

# for stacking regressor model
from mlxtend.regressor import StackingCVRegressor

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Read the Data

In [2]:
life_expectancy_data = pd.read_csv(r'C:\Users\Shahbaz\Desktop\socool\exam\regression\Life Expectancy Data.csv')
data = life_expectancy_data.copy()

In [3]:
data.head()

,Country,Year,Status,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,under-five deaths,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling,Life expectancy
0,Afghanistan,2015,Developing,263.0,62,0.01,71.279624,65.0,1154,19.1,83,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1,65.0
1,Afghanistan,2014,Developing,271.0,64,0.01,73.523582,62.0,492,18.6,86,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0,59.9
2,Afghanistan,2013,Developing,268.0,66,0.01,73.219243,64.0,430,18.1,89,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9,59.9
3,Afghanistan,2012,Developing,272.0,69,0.01,78.184215,67.0,2787,17.6,93,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8,59.5
4,Afghanistan,2011,Developing,275.0,71,0.01,7.097109,68.0,3013,17.2,97,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5,59.2


In [4]:
def accurate_columns(data):
    new_columns = []
    for col in data.columns:
        col = col.strip()
        if col[0].islower():
            col = col.capitalize()
        new_columns.append(col)
    data.columns = new_columns
    return data

In [5]:
data = accurate_columns(data)

In [6]:
data.head()

,Country,Year,Status,Adult Mortality,Infant deaths,Alcohol,Percentage expenditure,Hepatitis B,Measles,BMI,Under-five deaths,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,Thinness 1-19 years,Thinness 5-9 years,Income composition of resources,Schooling,Life expectancy
0,Afghanistan,2015,Developing,263.0,62,0.01,71.279624,65.0,1154,19.1,83,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1,65.0
1,Afghanistan,2014,Developing,271.0,64,0.01,73.523582,62.0,492,18.6,86,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0,59.9
2,Afghanistan,2013,Developing,268.0,66,0.01,73.219243,64.0,430,18.1,89,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9,59.9
3,Afghanistan,2012,Developing,272.0,69,0.01,78.184215,67.0,2787,17.6,93,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8,59.5
4,Afghanistan,2011,Developing,275.0,71,0.01,7.097109,68.0,3013,17.2,97,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5,59.2


### Handling missing values

In [7]:
def fill_num_columns(data, target):
    data = data.dropna(subset=[target], axis=0)
    
    columns_with_nulls = data.isnull().sum()
    columns_with_nulls = columns_with_nulls[columns_with_nulls > 0]
    null_df = pd.DataFrame({
        'Column': columns_with_nulls.index,
        'Null Count': columns_with_nulls.values,
        'Dtype': data[columns_with_nulls.index].dtypes.values
    })
    
    # Set threshold for iterative imputation
    threshold = data.shape[0] * 0.1
    
    # Separate columns for mean imputation and iterative imputation
    columns_mean_impute = null_df.loc[null_df['Null Count'] <= threshold, 'Column'].tolist()
    columns_iterative_impute = null_df.loc[null_df['Null Count'] > threshold, 'Column'].tolist()
    
    # Numeric columns for imputation
    data_filled = data.copy()
    
    # Impute missing values with mean for columns with low missing counts
    mean_imputer = SimpleImputer(strategy='mean')
    data_filled.loc[:, columns_mean_impute] = mean_imputer.fit_transform(data_filled.loc[:, columns_mean_impute])
    
    # Impute missing values iteratively for columns with high missing counts
    iterative_imputer = IterativeImputer(random_state=42, max_iter=10)
    data_filled.loc[:, columns_iterative_impute] = iterative_imputer.fit_transform(data_filled.loc[:, columns_iterative_impute])
    
    return data_filled


### Outlier treatment

In [8]:
def outlier_treat(data):
    numeric_data = data.select_dtypes(exclude='object')

    q1 = numeric_data.quantile(0.25)
    q3 = numeric_data.quantile(0.75)
    IQR = q3 - q1

    lower = q1 - 1.5 * IQR
    upper = q3 + 1.5 * IQR

    for i in data.columns:
        if data[i].dtypes != object:
            data[i] = np.where(data[i]<lower[i], lower[i], data[i])
            data[i] = np.where(data[i]>upper[i], upper[i], data[i])
    return data

In [9]:
def show_outliers(data):
    for i in data.columns:
        if data[i].dtypes != object:
            sns.boxplot(x=data[i], data=data)
            plt.title(f'Boxplot of {i}')
            plt.show()

### Convert Country to Numeric
- there I found 2 methods but the second one worked better

In [10]:
# !pip install geonamescache

In [11]:
# !pip install pycountry_convert

In [12]:
def convert_country(df):
    country_to_continent = {}

    for country_name in df['Country'].unique():
        try:
            # Try to get ISO alpha-2 code for the country name
            country_alpha2 = pc.country_name_to_country_alpha2(country_name)
            # Get continent code based on the ISO alpha-2 code
            continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
            country_to_continent[country_name] = continent_code
            
        except (KeyError, ValueError) as e:
            for x in country_name.split():
                try:
                    country_alpha2 = pc.country_name_to_country_alpha2(x)
                    # Get continent code based on the ISO alpha-2 code
                    continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
                    country_to_continent[country_name] = continent_code
                except:
                    # print(f"Error mapping country '{country_name}' to continent: {e}")
                    country_to_continent[country_name] = 'unknown'
            
    # Map the Continent based on the updated dictionary
    df['Continent'] = df['Country'].map(country_to_continent)
    df = df.drop('Country', axis=1)
    return df

### Convert Categoric to Numeric

In [13]:
def cat_to_num(data):
    data = data.fillna('missing')
    data = pd.get_dummies(data, drop_first=True).astype(int)
    return data

### Creating new features

In [14]:
def feature_engineering(data):
    stat_methods = {
        'mean': np.mean,
        'median': np.median,
        'min': np.min,
        'max': np.max,
        'std': np.std,
        'var': np.var
    }
    
    # Select calculation columns (excluding grouping columns)
    calculation_cols = [x for x in data.columns if x not in ['Continent', 'Year', 'Status']]
    result_data = data.copy()
    column_groups = {
        ('Continent',): calculation_cols,
        ('Year',): calculation_cols,
        ('Status',): calculation_cols,
        ('Continent', 'Year'): calculation_cols,
        ('Continent', 'Status'): calculation_cols,
        ('Status', 'Year'): calculation_cols
    }
    
    for stat_name, stat_func in stat_methods.items():
        for group_cols, calc_cols in column_groups.items():
            print(f"Processing: {stat_name}, Grouping: {group_cols}")
            
            grouped_data = data.groupby(list(group_cols))[calc_cols].agg(stat_func).reset_index()
            
            agg_col_names = [f'{col}_{stat_name}_by_' + '_'.join(group_cols) for col in calc_cols]
            grouped_data.columns = list(group_cols) + agg_col_names
            
            # Merge aggregated data back into the result_data dataframe on group_cols
            result_data = pd.merge(result_data, grouped_data, on=list(group_cols), how='left')
    
    return result_data


### Getting columns which is correlated with target

In [15]:
def target_correlation(data, target, threshold_normality=0.05, threshold_correlation=0.45):
    normal_dist = []
    not_normal_dist = []

    # Identify columns based on normality
    for col in data.columns:
        if data[col].dtype in ['int64', 'float64']:
            kstest_statistic, kstest_p_value = kstest(data[col], 'norm')
            if kstest_p_value > threshold_normality:
                normal_dist.append(col)
            else:
                not_normal_dist.append(col)

    # Calculate Spearman correlation with target for non-normally distributed columns
    corr_matrix = data.corr(method='spearman', numeric_only=True)
    variables_explaining_target = []

    for col in not_normal_dist:
        if col != target:
            spearman_corr = corr_matrix.loc[col, target]
            if abs(spearman_corr) > threshold_correlation:
                variables_explaining_target.append(col)

    # Include target, numeric columns explaining target, and non-numeric columns in the output
    result_cols = variables_explaining_target + [target] + list(data.select_dtypes(exclude='number').columns)
    
    return result_cols

# Intercorrelation check

In [16]:
def intercorrelation(data, target, threshold=0.5):
    normal_dist = []
    not_normal_dist = []

    for col in data.columns:
        if data[col].dtype in ['int64', 'float64']:
            kstest_statistic, kstest_p_value = kstest(data[col], 'norm')
            if kstest_p_value > threshold:
                normal_dist.append(col)
            else:
                not_normal_dist.append(col)
                
    for col in not_normal_dist:
        if col != target:
            corr_matrix = data.drop(columns=[target]).corr(method='spearman', numeric_only=True)
            
            low_correlated_variables = (np.abs(corr_matrix) < threshold) & (corr_matrix != 1.0)
            
            independent_variable_pairs = np.where(low_correlated_variables)
                
            not_intercorrelated_independent = pd.DataFrame({
                'Variable': corr_matrix.index[independent_variable_pairs[0]]
            })
    return not_intercorrelated_independent['Variable'].unique().tolist() + list(data.select_dtypes(exclude='number').columns)

### Scaling

In [17]:
def scale_data(data_train, data_test=None):
    scaler = StandardScaler()
    scaled_train = scaler.fit_transform(data_train)
    scaled_train_df = pd.DataFrame(scaled_train, columns=data_train.columns, index=data_train.index)

    if data_test is not None:
        scaled_test = scaler.transform(data_test)
        scaled_test_df = pd.DataFrame(scaled_test, columns=data_test.columns, index=data_test.index)
        return scaled_train_df, scaled_test_df
    else:
        return scaled_train_df


In [18]:
def calculate_vif(X):
    vif_data = pd.DataFrame()
    vif_data["Feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif_data

def vif_selection(data, threshold=5.0):
    non_numeric_cols = data.select_dtypes(exclude='number').columns.tolist()
    X = data.select_dtypes(include='number').copy()
    features_to_keep = list(X.columns)
    vif_data = calculate_vif(X)
    
    while vif_data['VIF'].max() > threshold:
        max_vif_feature = vif_data.loc[vif_data['VIF'].idxmax(), 'Feature']
        
        X = X.drop(max_vif_feature, axis=1)
        features_to_keep.remove(max_vif_feature)
        vif_data = calculate_vif(X)
        
        print(f'Removed feature: {max_vif_feature} with VIF: {vif_data["VIF"].max()}')
        print(f'Remaining features: {len(X.columns)}')
    
    final_vif_data = pd.DataFrame({
        'Feature': X.columns,
        'VIF': vif_data['VIF']
    })
    
    final_features = final_vif_data['Feature'].tolist() + non_numeric_cols
    
    return final_features


# Train and Test split

In [20]:
# linear regression data:
lr_data = fill_num_columns(data, 'Life expectancy')
lr_data = outlier_treat(lr_data)
lr_data = convert_country(lr_data)
lr_data = feature_engineering(lr_data)
lr_y = lr_data['Life expectancy'] # for all target
lr_data = lr_data[target_correlation(lr_data, 'Life expectancy')]
lr_data = lr_data[intercorrelation(lr_data, 'Life expectancy')]
svr_data = lr_data.copy() # the same steps
# lr_data = lr_data[vif_selection(lr_data, 7)] # it take so much time that's why I didn't run
lr_data = cat_to_num(lr_data)
X_train_lr, X_test_lr, y_train_lr, y_test_lr = train_test_split(lr_data, lr_y, random_state=42, test_size = 0.2)
X_train_lr, X_test_lr = scale_data(X_train_lr, X_test_lr)

Processing: mean, Grouping: ('Continent',)
Processing: mean, Grouping: ('Year',)
Processing: mean, Grouping: ('Status',)
Processing: mean, Grouping: ('Continent', 'Year')
Processing: mean, Grouping: ('Continent', 'Status')
Processing: mean, Grouping: ('Status', 'Year')
Processing: median, Grouping: ('Continent',)
Processing: median, Grouping: ('Year',)
Processing: median, Grouping: ('Status',)
Processing: median, Grouping: ('Continent', 'Year')
Processing: median, Grouping: ('Continent', 'Status')
Processing: median, Grouping: ('Status', 'Year')
Processing: min, Grouping: ('Continent',)
Processing: min, Grouping: ('Year',)
Processing: min, Grouping: ('Status',)
Processing: min, Grouping: ('Continent', 'Year')
Processing: min, Grouping: ('Continent', 'Status')
Processing: min, Grouping: ('Status', 'Year')
Processing: max, Grouping: ('Continent',)
Processing: max, Grouping: ('Year',)


C:\Users\Shahbaz\AppData\Local\Temp\ipykernel_4300\3285829363.py:27: FutureWarning: The provided callable <function mean at 0x000002095C280D60> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  grouped_data = data.groupby(list(group_cols))[calc_cols].agg(stat_func).reset_index()
C:\Users\Shahbaz\AppData\Local\Temp\ipykernel_4300\3285829363.py:27: FutureWarning: The provided callable <function mean at 0x000002095C280D60> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  grouped_data = data.groupby(list(group_cols))[calc_cols].agg(stat_func).reset_index()
C:\Users\Shahbaz\AppData\Local\Temp\ipykernel_4300\3285829363.py:27: FutureWarning: The provided callable <function mean at 0x000002095C280D60> is currently using DataFrameGroupBy.mean. 

Processing: max, Grouping: ('Status',)
Processing: max, Grouping: ('Continent', 'Year')
Processing: max, Grouping: ('Continent', 'Status')
Processing: max, Grouping: ('Status', 'Year')
Processing: std, Grouping: ('Continent',)
Processing: std, Grouping: ('Year',)
Processing: std, Grouping: ('Status',)
Processing: std, Grouping: ('Continent', 'Year')
Processing: std, Grouping: ('Continent', 'Status')
Processing: std, Grouping: ('Status', 'Year')
Processing: var, Grouping: ('Continent',)
Processing: var, Grouping: ('Year',)


C:\Users\Shahbaz\AppData\Local\Temp\ipykernel_4300\3285829363.py:27: FutureWarning: The provided callable <function max at 0x000002095C280360> is currently using DataFrameGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  grouped_data = data.groupby(list(group_cols))[calc_cols].agg(stat_func).reset_index()
C:\Users\Shahbaz\AppData\Local\Temp\ipykernel_4300\3285829363.py:27: FutureWarning: The provided callable <function max at 0x000002095C280360> is currently using DataFrameGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  grouped_data = data.groupby(list(group_cols))[calc_cols].agg(stat_func).reset_index()
C:\Users\Shahbaz\AppData\Local\Temp\ipykernel_4300\3285829363.py:27: FutureWarning: The provided callable <function max at 0x000002095C280360> is currently using DataFrameGroupBy.max. In a fut

Processing: var, Grouping: ('Status',)
Processing: var, Grouping: ('Continent', 'Year')
Processing: var, Grouping: ('Continent', 'Status')
Processing: var, Grouping: ('Status', 'Year')


C:\Users\Shahbaz\AppData\Local\Temp\ipykernel_4300\3285829363.py:27: FutureWarning: The provided callable <function var at 0x000002095C280FE0> is currently using DataFrameGroupBy.var. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "var" instead.
  grouped_data = data.groupby(list(group_cols))[calc_cols].agg(stat_func).reset_index()
C:\Users\Shahbaz\AppData\Local\Temp\ipykernel_4300\3285829363.py:27: FutureWarning: The provided callable <function var at 0x000002095C280FE0> is currently using DataFrameGroupBy.var. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "var" instead.
  grouped_data = data.groupby(list(group_cols))[calc_cols].agg(stat_func).reset_index()
C:\Users\Shahbaz\AppData\Local\Temp\ipykernel_4300\3285829363.py:27: FutureWarning: The provided callable <function var at 0x000002095C280FE0> is currently using DataFrameGroupBy.var. In a fut

In [21]:
X_train_lr.head()

,Adult Mortality,Infant deaths,BMI,Under-five deaths,Polio,Diphtheria,HIV/AIDS,GDP,Thinness 1-19 years,Thinness 5-9 years,Income composition of resources,Schooling,Adult Mortality_mean_by_Continent,Infant deaths_mean_by_Continent,Percentage expenditure_mean_by_Continent,Hepatitis B_mean_by_Continent,Measles_mean_by_Continent,BMI_mean_by_Continent,Under-five deaths_mean_by_Continent,Polio_mean_by_Continent,Diphtheria_mean_by_Continent,HIV/AIDS_mean_by_Continent,GDP_mean_by_Continent,Thinness 1-19 years_mean_by_Continent,Thinness 5-9 years_mean_by_Continent,Income composition of resources_mean_by_Continent,Schooling_mean_by_Continent,Life expectancy_mean_by_Continent,Adult Mortality_mean_by_Status,Infant deaths_mean_by_Status,Alcohol_mean_by_Status,Percentage expenditure_mean_by_Status,Hepatitis B_mean_by_Status,Measles_mean_by_Status,BMI_mean_by_Status,Under-five deaths_mean_by_Status,Polio_mean_by_Status,Total expenditure_mean_by_Status,Diphtheria_mean_by_Status,HIV/AIDS_mean_by_Status,GDP_mean_by_Status,Population_mean_by_Status,Thinness 1-19 years_mean_by_Status,Thinness 5-9 years_mean_by_Status,Income composition of resources_mean_by_Status,Schooling_mean_by_Status,Life expectancy_mean_by_Status,Adult Mortality_mean_by_Continent_Year,Infant deaths_mean_by_Continent_Year,Percentage expenditure_mean_by_Continent_Year,Hepatitis B_mean_by_Continent_Year,BMI_mean_by_Continent_Year,Under-five deaths_mean_by_Continent_Year,Polio_mean_by_Continent_Year,Diphtheria_mean_by_Continent_Year,HIV/AIDS_mean_by_Continent_Year,GDP_mean_by_Continent_Year,Thinness 1-19 years_mean_by_Continent_Year,Thinness 5-9 years_mean_by_Continent_Year,Income composition of resources_mean_by_Continent_Year,Schooling_mean_by_Continent_Year,Life expectancy_mean_by_Continent_Year,Adult Mortality_mean_by_Continent_Status,Infant deaths_mean_by_Continent_Status,Alcohol_mean_by_Continent_Status,Percentage expenditure_mean_by_Continent_Status,Hepatitis B_mean_by_Continent_Status,BMI_mean_by_Continent_Status,Under-five deaths_mean_by_Continent_Status,Polio_mean_by_Continent_Status,Total expenditure_mean_by_Continent_Status,Diphtheria_mean_by_Continent_Status,HIV/AIDS_mean_by_Continent_Status,GDP_mean_by_Continent_Status,Thinness 1-19 years_mean_by_Continent_Status,Thinness 5-9 years_mean_by_Continent_Status,Income composition of resources_mean_by_Continent_Status,Schooling_mean_by_Continent_Status,Life expectancy_mean_by_Continent_Status,Adult Mortality_mean_by_Status_Year,Infant deaths_mean_by_Status_Year,BMI_mean_by_Status_Year,Under-five deaths_mean_by_Status_Year,Diphtheria_mean_by_Status_Year,HIV/AIDS_mean_by_Status_Year,GDP_mean_by_Status_Year,Thinness 1-19 years_mean_by_Status_Year,Thinness 5-9 years_mean_by_Status_Year,Income composition of resources_mean_by_Status_Year,Schooling_mean_by_Status_Year,Life expectancy_mean_by_Status_Year,Adult Mortality_median_by_Continent,Infant deaths_median_by_Continent,Percentage expenditure_median_by_Continent,Hepatitis B_median_by_Continent,Measles_median_by_Continent,BMI_median_by_Continent,Under-five deaths_median_by_Continent,Polio_median_by_Continent,Diphtheria_median_by_Continent,HIV/AIDS_median_by_Continent,GDP_median_by_Continent,Thinness 1-19 years_median_by_Continent,Thinness 5-9 years_median_by_Continent,Income composition of resources_median_by_Continent,Schooling_median_by_Continent,Life expectancy_median_by_Continent,Adult Mortality_median_by_Status,Infant deaths_median_by_Status,Alcohol_median_by_Status,Percentage expenditure_median_by_Status,Hepatitis B_median_by_Status,Measles_median_by_Status,BMI_median_by_Status,Under-five deaths_median_by_Status,Polio_median_by_Status,Total expenditure_median_by_Status,Diphtheria_median_by_Status,GDP_median_by_Status,Population_median_by_Status,Thinness 1-19 years_median_by_Status,Thinness 5-9 years_median_by_Status,Income composition of resources_median_by_Status,Schooling_median_by_Status,Life expectancy_median_by_Status,Adult Mortality_median_by_Continent_Year,Infant

In [23]:
lr_data.head()

,Adult Mortality,Infant deaths,BMI,Under-five deaths,Polio,Diphtheria,HIV/AIDS,GDP,Thinness 1-19 years,Thinness 5-9 years,Income composition of resources,Schooling,Adult Mortality_mean_by_Continent,Infant deaths_mean_by_Continent,Percentage expenditure_mean_by_Continent,Hepatitis B_mean_by_Continent,Measles_mean_by_Continent,BMI_mean_by_Continent,Under-five deaths_mean_by_Continent,Polio_mean_by_Continent,Diphtheria_mean_by_Continent,HIV/AIDS_mean_by_Continent,GDP_mean_by_Continent,Thinness 1-19 years_mean_by_Continent,Thinness 5-9 years_mean_by_Continent,Income composition of resources_mean_by_Continent,Schooling_mean_by_Continent,Life expectancy_mean_by_Continent,Adult Mortality_mean_by_Status,Infant deaths_mean_by_Status,Alcohol_mean_by_Status,Percentage expenditure_mean_by_Status,Hepatitis B_mean_by_Status,Measles_mean_by_Status,BMI_mean_by_Status,Under-five deaths_mean_by_Status,Polio_mean_by_Status,Total expenditure_mean_by_Status,Diphtheria_mean_by_Status,HIV/AIDS_mean_by_Status,GDP_mean_by_Status,Population_mean_by_Status,Thinness 1-19 years_mean_by_Status,Thinness 5-9 years_mean_by_Status,Income composition of resources_mean_by_Status,Schooling_mean_by_Status,Life expectancy_mean_by_Status,Adult Mortality_mean_by_Continent_Year,Infant deaths_mean_by_Continent_Year,Percentage expenditure_mean_by_Continent_Year,Hepatitis B_mean_by_Continent_Year,BMI_mean_by_Continent_Year,Under-five deaths_mean_by_Continent_Year,Polio_mean_by_Continent_Year,Diphtheria_mean_by_Continent_Year,HIV/AIDS_mean_by_Continent_Year,GDP_mean_by_Continent_Year,Thinness 1-19 years_mean_by_Continent_Year,Thinness 5-9 years_mean_by_Continent_Year,Income composition of resources_mean_by_Continent_Year,Schooling_mean_by_Continent_Year,Life expectancy_mean_by_Continent_Year,Adult Mortality_mean_by_Continent_Status,Infant deaths_mean_by_Continent_Status,Alcohol_mean_by_Continent_Status,Percentage expenditure_mean_by_Continent_Status,Hepatitis B_mean_by_Continent_Status,BMI_mean_by_Continent_Status,Under-five deaths_mean_by_Continent_Status,Polio_mean_by_Continent_Status,Total expenditure_mean_by_Continent_Status,Diphtheria_mean_by_Continent_Status,HIV/AIDS_mean_by_Continent_Status,GDP_mean_by_Continent_Status,Thinness 1-19 years_mean_by_Continent_Status,Thinness 5-9 years_mean_by_Continent_Status,Income composition of resources_mean_by_Continent_Status,Schooling_mean_by_Continent_Status,Life expectancy_mean_by_Continent_Status,Adult Mortality_mean_by_Status_Year,Infant deaths_mean_by_Status_Year,BMI_mean_by_Status_Year,Under-five deaths_mean_by_Status_Year,Diphtheria_mean_by_Status_Year,HIV/AIDS_mean_by_Status_Year,GDP_mean_by_Status_Year,Thinness 1-19 years_mean_by_Status_Year,Thinness 5-9 years_mean_by_Status_Year,Income composition of resources_mean_by_Status_Year,Schooling_mean_by_Status_Year,Life expectancy_mean_by_Status_Year,Adult Mortality_median_by_Continent,Infant deaths_median_by_Continent,Percentage expenditure_median_by_Continent,Hepatitis B_median_by_Continent,Measles_median_by_Continent,BMI_median_by_Continent,Under-five deaths_median_by_Continent,Polio_median_by_Continent,Diphtheria_median_by_Continent,HIV/AIDS_median_by_Continent,GDP_median_by_Continent,Thinness 1-19 years_median_by_Continent,Thinness 5-9 years_median_by_Continent,Income composition of resources_median_by_Continent,Schooling_median_by_Continent,Life expectancy_median_by_Continent,Adult Mortality_median_by_Status,Infant deaths_median_by_Status,Alcohol_median_by_Status,Percentage expenditure_median_by_Status,Hepatitis B_median_by_Status,Measles_median_by_Status,BMI_median_by_Status,Under-five deaths_median_by_Status,Polio_median_by_Status,Total expenditure_median_by_Status,Diphtheria_median_by_Status,GDP_median_by_Status,Population_median_by_Status,Thinness 1-19 years_median_by_Status,Thinness 5-9 years_median_by_Status,Income composition of resources_median_by_Status,Schooling_median_by_Status,Life expectancy_median_by_Status,Adult Mortality_median_by_Continent_Year,Infant

In [24]:
# decision tree data and random forest:
dt_data = fill_num_columns(data, 'Life expectancy')
dt_data = convert_country(dt_data)
dt_data = feature_engineering(dt_data)
dt_data = cat_to_num(dt_data)
dt_data = dt_data.drop('Life expectancy', axis=1)
X_train_tr, X_test_tr, y_train_tr, y_test_tr = train_test_split(dt_data, lr_y, random_state=42, test_size = 0.2)

Processing: mean, Grouping: ('Continent',)
Processing: mean, Grouping: ('Year',)
Processing: mean, Grouping: ('Status',)
Processing: mean, Grouping: ('Continent', 'Year')
Processing: mean, Grouping: ('Continent', 'Status')
Processing: mean, Grouping: ('Status', 'Year')
Processing: median, Grouping: ('Continent',)
Processing: median, Grouping: ('Year',)
Processing: median, Grouping: ('Status',)
Processing: median, Grouping: ('Continent', 'Year')
Processing: median, Grouping: ('Continent', 'Status')
Processing: median, Grouping: ('Status', 'Year')
Processing: min, Grouping: ('Continent',)
Processing: min, Grouping: ('Year',)
Processing: min, Grouping: ('Status',)
Processing: min, Grouping: ('Continent', 'Year')
Processing: min, Grouping: ('Continent', 'Status')
Processing: min, Grouping: ('Status', 'Year')
Processing: max, Grouping: ('Continent',)


C:\Users\Shahbaz\AppData\Local\Temp\ipykernel_4300\3285829363.py:27: FutureWarning: The provided callable <function mean at 0x000002095C280D60> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  grouped_data = data.groupby(list(group_cols))[calc_cols].agg(stat_func).reset_index()
C:\Users\Shahbaz\AppData\Local\Temp\ipykernel_4300\3285829363.py:27: FutureWarning: The provided callable <function mean at 0x000002095C280D60> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  grouped_data = data.groupby(list(group_cols))[calc_cols].agg(stat_func).reset_index()
C:\Users\Shahbaz\AppData\Local\Temp\ipykernel_4300\3285829363.py:27: FutureWarning: The provided callable <function mean at 0x000002095C280D60> is currently using DataFrameGroupBy.mean. 

Processing: max, Grouping: ('Year',)
Processing: max, Grouping: ('Status',)
Processing: max, Grouping: ('Continent', 'Year')
Processing: max, Grouping: ('Continent', 'Status')
Processing: max, Grouping: ('Status', 'Year')
Processing: std, Grouping: ('Continent',)
Processing: std, Grouping: ('Year',)
Processing: std, Grouping: ('Status',)
Processing: std, Grouping: ('Continent', 'Year')
Processing: std, Grouping: ('Continent', 'Status')
Processing: std, Grouping: ('Status', 'Year')
Processing: var, Grouping: ('Continent',)
Processing: var, Grouping: ('Year',)


C:\Users\Shahbaz\AppData\Local\Temp\ipykernel_4300\3285829363.py:27: FutureWarning: The provided callable <function max at 0x000002095C280360> is currently using DataFrameGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  grouped_data = data.groupby(list(group_cols))[calc_cols].agg(stat_func).reset_index()
C:\Users\Shahbaz\AppData\Local\Temp\ipykernel_4300\3285829363.py:27: FutureWarning: The provided callable <function max at 0x000002095C280360> is currently using DataFrameGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  grouped_data = data.groupby(list(group_cols))[calc_cols].agg(stat_func).reset_index()
C:\Users\Shahbaz\AppData\Local\Temp\ipykernel_4300\3285829363.py:27: FutureWarning: The provided callable <function max at 0x000002095C280360> is currently using DataFrameGroupBy.max. In a fut

Processing: var, Grouping: ('Status',)
Processing: var, Grouping: ('Continent', 'Year')
Processing: var, Grouping: ('Continent', 'Status')
Processing: var, Grouping: ('Status', 'Year')


C:\Users\Shahbaz\AppData\Local\Temp\ipykernel_4300\3285829363.py:27: FutureWarning: The provided callable <function var at 0x000002095C280FE0> is currently using DataFrameGroupBy.var. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "var" instead.
  grouped_data = data.groupby(list(group_cols))[calc_cols].agg(stat_func).reset_index()
C:\Users\Shahbaz\AppData\Local\Temp\ipykernel_4300\3285829363.py:27: FutureWarning: The provided callable <function var at 0x000002095C280FE0> is currently using DataFrameGroupBy.var. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "var" instead.
  grouped_data = data.groupby(list(group_cols))[calc_cols].agg(stat_func).reset_index()
C:\Users\Shahbaz\AppData\Local\Temp\ipykernel_4300\3285829363.py:27: FutureWarning: The provided callable <function var at 0x000002095C280FE0> is currently using DataFrameGroupBy.var. In a fut

In [25]:
# svr data:
svr_data = cat_to_num(svr_data)
X_train_svr, X_test_svr, y_train_svr, y_test_svr = train_test_split(svr_data, lr_y, random_state=42, test_size = 0.2)
X_train_svr, X_test_svr = scale_data(X_train_svr, X_test_svr)

In [26]:
# xgb lgbm catboost:
boost_data = convert_country(data)
boost_data = boost_data.dropna(subset = ['Life expectancy'], axis=0)
boost_data_custom = boost_data.copy() # same step for custom data
boost_data = boost_data.select_dtypes(include='number')
boost_data = boost_data.drop('Life expectancy', axis=1)
X_train_boost, X_test_boost, y_train_boost, y_test_boost = train_test_split(boost_data, lr_y, random_state=42, test_size = 0.2)

In [27]:
# custom catboost:
columns_to_fill = boost_data_custom.select_dtypes(exclude='number').columns.tolist()
boost_data_custom[columns_to_fill] = boost_data_custom[columns_to_fill].fillna('Missing Value')
boost_data_custom = boost_data_custom.drop('Life expectancy', axis=1)
X_train_cat, X_test_cat, y_train_cat, y_test_cat = train_test_split(boost_data_custom, lr_y, random_state=42, test_size = 0.2)

# Importing the Models

In [28]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

# Train and Evaluate

In [29]:
default_models = []

lin_reg_def = LinearRegression()
dec_tree_def = DecisionTreeRegressor()
rfr_def = RandomForestRegressor()
svr_def = SVR()
xgboost_def = XGBRegressor()
lgbm_def = LGBMRegressor()
catboost_def = CatBoostRegressor()
catboost_custom_def = CatBoostRegressor(cat_features=[x for x in X_train_cat.columns if (X_train_cat[x].dtype == object)])

default_models.extend([
    ('LinearRegression', lin_reg_def),
    ('DecisionTreeRegressor', dec_tree_def),
    ('RFR', rfr_def),
    ('SVR', svr_def),
    ('XGB', xgboost_def), 
    ('LightGBM', lgbm_def),
    ('CatBoost', catboost_def),
    ('Custom_CatBoost_Cat', catboost_custom_def)
])

In [30]:
def train_test_and_evaluate_model(model_name, model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    r2_train = r2_score(y_train, y_pred_train)
    r2_test = r2_score(y_test, y_pred_test)
    
    return r2_train, r2_test

In [31]:
def evaluate_models(models):
    r2_df = pd.DataFrame(columns=['Model', 'R2_train', 'R2_test'])
    
    for model_name, model in models:
        if model_name.startswith('LinearRegression'):
            r2_train, r2_test = train_test_and_evaluate_model(model_name, model, X_train_lr, y_train_lr, X_test_lr, y_test_lr)
        
        elif model_name.startswith('DecisionTreeRegressor') or model_name.startswith('RFR'):
            r2_train, r2_test = train_test_and_evaluate_model(model_name, model, X_train_tr, y_train_tr, X_test_tr, y_test_tr)
        
        elif model_name.startswith('Custom_CatBoost_Cat'):
            r2_train, r2_test = train_test_and_evaluate_model(model_name, model, X_train_cat, y_train_cat, X_test_cat, y_test_cat)
        
        elif model_name.startswith('SVR'):
            r2_train, r2_test = train_test_and_evaluate_model(model_name, model, X_train_svr, y_train_svr, X_test_svr, y_test_svr)
        
        elif model_name.startswith('XGB') or model_name.startswith('LightGBM') or model_name.startswith('CatBoost'):
            r2_train, r2_test = train_test_and_evaluate_model(model_name, model, X_train_boost, y_train_boost, X_test_boost, y_test_boost)
        
        if r2_train is not None:
            r2_df = pd.concat([r2_df, pd.DataFrame({'Model': [model_name], 'R2_train': [r2_train], 'R2_test': [r2_test]})], ignore_index=True)
    
    r2_df_sorted = r2_df.sort_values(by='R2_train', ascending=False)
    return r2_df_sorted

In [32]:
default_model_result = evaluate_models(default_models)

C:\Users\Shahbaz\AppData\Local\Temp\ipykernel_4300\1535037803.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  r2_df = pd.concat([r2_df, pd.DataFrame({'Model': [model_name], 'R2_train': [r2_train], 'R2_test': [r2_test]})], ignore_index=True)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000096 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3360
[LightGBM] [Info] Number of data points in the train set: 2342, number of used features: 19
[LightGBM] [Info] Start training from score 69.158369
Learning rate set to 0.046835
0:	learn: 9.2080057	total: 147ms	remaining: 2m 27s
1:	learn: 8.8833878	total: 150ms	remaining: 1m 14s
2:	learn: 8.5703469	total: 152ms	remaining: 50.4s
3:	learn: 8.2764824	total: 154ms	remaining: 38.3s
4:	learn: 7.9980043	total: 156ms	remaining: 31s
5:	learn: 7.7281368	total: 158ms	remaining: 26.2s
6:	learn: 7.4786121	total: 161ms	remaining: 22.8s
7:	learn: 7.2256969	total: 163ms	remaining: 20.2s
8:	learn: 6.9992185	total: 165ms	remaining: 18.2s
9:	learn: 6.7745048	total: 168ms	r

In [33]:
default_model_result

,Model,R2_train,R2_test
1,DecisionTreeRegressor,1.000000,0.913922
4,XGB,0.999538,0.967510
2,RFR,0.994133,0.958910
6,CatBoost,0.993469,0.968466
7,Custom_CatBoost_Cat,0.992841,0.968598
5,LightGBM,0.991372,0.966060
0,LinearRegression,0.856166,0.851753
3,SVR,0.822095,0.823842


- using custom CatBoost doesn't change score because there wasn't any missing value in categorical columns

# Optimization
- #### With Optuna

In [34]:
def best_params_for_model(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 1000),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'num_leaves': trial.suggest_int('num_leaves', 10, 100)
    }
    lgb_reg = LGBMRegressor(**param)
    
    r2 = cross_val_score(lgb_reg, X_train_boost, y_train_boost, cv=3, scoring='r2', n_jobs=-1).mean()
    return r2

study = optuna.create_study(direction='maximize')
study.optimize(best_params_for_model, n_trials=3)

print('Best trial:')
best_params = study.best_params
print('  Value: {:.3f}'.format(study.best_value))
print('  Params: ', best_params)

best_lgb_model = LGBMRegressor(**best_params)


[I 2024-05-19 21:53:09,084] A new study created in memory with name: no-name-7a120936-f501-42ec-8074-0268da06f488
C:\Users\Shahbaz\AppData\Local\Temp\ipykernel_4300\3962178921.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),
[I 2024-05-19 21:53:11,211] Trial 0 finished with value: 0.9571454761016334 and parameters: {'n_estimators': 483, 'learning_rate': 0.0965786280281194, 'max_depth': 5, 'num_leaves': 42}. Best is trial 0 with value: 0.9571454761016334.
[I 2024-05-19 21:53:13,633] Trial 1 finished with value: 0.9586920800437649 and parameters: {'n_estimators': 925, 'learning_rate': 0.04016648230665793, 'max_depth': 8, 'num_leaves': 44}. Best is trial 1 with value: 0.9586920800437649.
[I 2024-05-19 21:53:14,938] Trial 2 finished with value: 0.948518347030

Best trial:
  Value: 0.959
  Params:  {'n_estimators': 925, 'learning_rate': 0.04016648230665793, 'max_depth': 8, 'num_leaves': 44}


In [35]:
def best_params_for_model(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 1000),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_uniform('subsample', 0.5,1),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5,1),
        'gamma': trial.suggest_int('gamma', 0,5)
        # Increasing the gamma value can help to prevent overfitting
        # Should be used only when you are using high depth
    }
    
    xgb_reg = XGBRegressor(**param)
    r2 = cross_val_score(xgb_reg, X_train_boost, y_train_boost, cv=3, scoring='r2', n_jobs=-1).mean()
    return r2

study = optuna.create_study(direction='maximize')
study.optimize(best_params_for_model, n_trials=3)

print('Best trial:')
best_params = study.best_params
print('  Value: {:.3f}'.format(study.best_value))
print('  Params: ', best_params)

best_xgb_model = XGBRegressor(**best_params)

[I 2024-05-19 21:53:14,957] A new study created in memory with name: no-name-7b79d159-c5bc-4f0b-86a0-7beaa00eb5c5
C:\Users\Shahbaz\AppData\Local\Temp\ipykernel_4300\853130857.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),
C:\Users\Shahbaz\AppData\Local\Temp\ipykernel_4300\853130857.py:6: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.5,1),
C:\Users\Shahbaz\AppData\Local\Temp\ipykernel_4300\853130857.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Us

Best trial:
  Value: 0.959
  Params:  {'n_estimators': 911, 'learning_rate': 0.043969998299955695, 'max_depth': 6, 'subsample': 0.5813636005820599, 'colsample_bytree': 0.7261432509435952, 'gamma': 5}


In [36]:
def best_params_for_model(trial):
    param = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),
        'depth': trial.suggest_int('depth', 3, 10),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 0.1, 10),
        'loss_function': trial.suggest_categorical('loss_function', ['RMSE']) # For regression tasks, use ‘RMSE’
    }
    cb_reg = CatBoostRegressor(**param, verbose=0)
    
    r2 = cross_val_score(cb_reg, X_train_boost, y_train_boost, cv=3, scoring='r2', n_jobs=-1).mean()
    return r2

study = optuna.create_study(direction='maximize')
study.optimize(best_params_for_model, n_trials=3)

print('Best trial:')
best_params = study.best_params
print('  Value: {:.3f}'.format(study.best_value))
print('  Params: ', best_params)

best_cb_model = CatBoostRegressor(**best_params)

[I 2024-05-19 21:53:16,940] A new study created in memory with name: no-name-3fcbd718-fe27-48dc-987c-b5c8b228e547
C:\Users\Shahbaz\AppData\Local\Temp\ipykernel_4300\4021564926.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),
C:\Users\Shahbaz\AppData\Local\Temp\ipykernel_4300\4021564926.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 0.1, 10),
[I 2024-05-19 21:53:25,403] Trial 0 finished with value: 0.951026934711844 and parameters: {'iterations': 274, 'learning_rate': 0.0407099380733281, 'depth': 9, 'l2_leaf_reg': 0.617487484310912, 'loss_

Best trial:
  Value: 0.961
  Params:  {'iterations': 855, 'learning_rate': 0.07415749522830317, 'depth': 7, 'l2_leaf_reg': 0.8557630258855302, 'loss_function': 'RMSE'}


In [37]:
def objective(trial):    
    params = {
        "n_estimators": trial.suggest_int(name="n_estimators", low=100, high=500, step=100),
        "max_features": trial.suggest_categorical(name="max_features", choices=['log2', 'sqrt']),
        "max_depth": trial.suggest_int(name="max_depth", low=10, high=110, step=20),
        "min_samples_split": trial.suggest_int(name="min_samples_split", low=2, high=10, step=2),
        "min_samples_leaf": trial.suggest_int(name="min_samples_leaf", low=1, high=4, step=1)
    }
    model = RandomForestRegressor(**params)
    
    r2 = cross_val_score(model, X_train_tr, y_train_tr, scoring='r2', n_jobs=-1, cv=5).mean()
    return np.mean(r2)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=5)

best_rfr_model = RandomForestRegressor(**study.best_params)

[I 2024-05-19 21:53:35,350] A new study created in memory with name: no-name-9cd3175c-a8fd-472f-9e35-eba78d4c8772
[I 2024-05-19 21:53:40,678] Trial 0 finished with value: 0.8068652682629638 and parameters: {'n_estimators': 500, 'max_features': 'log2', 'max_depth': 90, 'min_samples_split': 6, 'min_samples_leaf': 1}. Best is trial 0 with value: 0.8068652682629638.
[I 2024-05-19 21:53:41,810] Trial 1 finished with value: 0.7935527045882261 and parameters: {'n_estimators': 100, 'max_features': 'log2', 'max_depth': 70, 'min_samples_split': 8, 'min_samples_leaf': 2}. Best is trial 0 with value: 0.8068652682629638.
[I 2024-05-19 21:53:44,911] Trial 2 finished with value: 0.7841972336022871 and parameters: {'n_estimators': 400, 'max_features': 'log2', 'max_depth': 70, 'min_samples_split': 2, 'min_samples_leaf': 4}. Best is trial 0 with value: 0.8068652682629638.
[I 2024-05-19 21:53:46,563] Trial 3 finished with value: 0.8552016672591194 and parameters: {'n_estimators': 100, 'max_features': 'sq

In [38]:
def best_params_for_model(trial):
    param = {
        'C': trial.suggest_loguniform('C', 0.1, 10),  # Regularization parameter
        'kernel': trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid']),  # Kernel type
        'degree': trial.suggest_int('degree', 2, 5),  # Degree for polynomial kernel (only for 'poly' kernel)
        'gamma': trial.suggest_categorical('gamma', ['auto', 'scale']),  # Kernel coefficient for 'rbf', 'poly', and 'sigmoid'
    }
    
    svr = SVR(**param)
    
    score = cross_val_score(svr, X_train_svr, y_train_svr, cv=3, scoring='r2', n_jobs=-1).mean()
    return score

study = optuna.create_study(direction='maximize')
study.optimize(best_params_for_model, n_trials=2)

print('Best trial:')
best_params = study.best_params
print('  Value: {:.3f}'.format(study.best_value))
print('  Params: ', best_params)
best_svr_model = SVR(**best_params)

[I 2024-05-19 21:54:02,741] A new study created in memory with name: no-name-2e97d6df-fad3-4d92-bb0e-27c0c3ab59ec
C:\Users\Shahbaz\AppData\Local\Temp\ipykernel_4300\103135921.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'C': trial.suggest_loguniform('C', 0.1, 10),  # Regularization parameter
[I 2024-05-19 21:54:04,038] Trial 0 finished with value: 0.7926558565508248 and parameters: {'C': 3.7681584929043495, 'kernel': 'poly', 'degree': 4, 'gamma': 'scale'}. Best is trial 0 with value: 0.7926558565508248.
[I 2024-05-19 21:54:05,409] Trial 1 finished with value: 0.6660007003926531 and parameters: {'C': 0.6214608921373636, 'kernel': 'sigmoid', 'degree': 3, 'gamma': 'scale'}. Best is trial 0 with value: 0.7926558565508248.


Best trial:
  Value: 0.793
  Params:  {'C': 3.7681584929043495, 'kernel': 'poly', 'degree': 4, 'gamma': 'scale'}


### Optimized Models

In [39]:
models_optimized = []

models_optimized.extend([
    ('SVR Optuna', best_svr_model),
    ('XGBoost Optuna', best_xgb_model),
    ('LightGBM Optuna', best_lgb_model),
    ('CatBoost Optuna', best_cb_model),
    ('RFR Optuna', best_rfr_model)
])

In [40]:
optimized_model_result = evaluate_models(models_optimized)

C:\Users\Shahbaz\AppData\Local\Temp\ipykernel_4300\1535037803.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  r2_df = pd.concat([r2_df, pd.DataFrame({'Model': [model_name], 'R2_train': [r2_train], 'R2_test': [r2_test]})], ignore_index=True)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000234 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3360
[LightGBM] [Info] Number of data points in the train set: 2342, number of used features: 19
[LightGBM] [Info] Start training from score 69.158369
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive 

In [41]:
optimized_model_result

,Model,R2_train,R2_test
3,CatBoost Optuna,0.999012,0.969112
2,LightGBM Optuna,0.998689,0.968906
1,XGBoost Optuna,0.991498,0.969560
4,RFR Optuna,0.983608,0.884114
0,SVR Optuna,0.819527,0.813701


In [42]:
final_model_results = pd.concat([default_model_result, optimized_model_result], axis=0)
final_model_results = final_model_results.sort_values(by='R2_train', ascending=False)
final_model_results.reset_index(drop=True, inplace=True)
final_model_results

,Model,R2_train,R2_test
0,DecisionTreeRegressor,1.000000,0.913922
1,XGB,0.999538,0.967510
2,CatBoost Optuna,0.999012,0.969112
3,LightGBM Optuna,0.998689,0.968906
4,RFR,0.994133,0.958910
5,CatBoost,0.993469,0.968466
6,Custom_CatBoost_Cat,0.992841,0.968598
7,XGBoost Optuna,0.991498,0.969560
8,LightGBM,0.991372,0.966060
9,RFR Optuna,0.983608,0.884114


# Univariate analysis with default XGBoost

- no matter we have created new 300 features, at the end Boosting works better

In [43]:
def univariate_analysis(model, X_train, y_train, X_test, y_test):
    variables = []
    train_r2_scores = []
    test_r2_scores = []


    for i in X_train.columns:
        X_train_single_var = X_train[[i]]
        X_test_single_var = X_test[[i]]
        
        model.fit(X_train_single_var, y_train)
        y_pred_train_single_var = model.predict(X_train_single_var)
        train_r2 = r2_score(y_train, y_pred_train_single_var)
        
        y_pred_test_single_var = model.predict(X_test_single_var)
        test_r2 = r2_score(y_test, y_pred_test_single_var)
        
        variables.append(i)
        train_r2_scores.append(train_r2)
        test_r2_scores.append(test_r2)
        
    results_df = pd.DataFrame({
        'Variable': variables,
        'Train R2': train_r2_scores,
        'Test R2': test_r2_scores
    })
    results_df_sorted = results_df.sort_values(by='Train R2', ascending=False)
    pd.options.display.float_format = '{:.4f}'.format
    return results_df_sorted

In [44]:
best_xgb_model_analysis = univariate_analysis(best_xgb_model, X_train_boost, y_train_boost, X_test_boost, y_test_boost)

In [45]:
best_xgb_model_analysis

,Variable,Train R2,Test R2
1,Adult Mortality,0.8170,0.8374
17,Income composition of resources,0.7247,0.6498
12,HIV/AIDS,0.6867,0.6588
18,Schooling,0.6058,0.5733
7,BMI,0.6032,0.4580
16,Thinness 5-9 years,0.5307,0.3905
8,Under-five deaths,0.5264,0.3563
15,Thinness 1-19 years,0.5240,0.3893
2,Infant deaths,0.4616,0.3873
11,Diphtheria,0.4342,0.3958


In [46]:
final_xgb_model_input = best_xgb_model_analysis['Variable'].tolist()[:4]
final_xgb_model_train = X_train_boost[final_xgb_model_input]
final_xgb_model_test = X_test_boost[final_xgb_model_input]

In [47]:
best_xgb_model_final = best_xgb_model.fit(final_xgb_model_train, y_train_boost)

In [48]:
final_r2_train, final_r2_test = train_test_and_evaluate_model('Final XGB', best_xgb_model_final, final_xgb_model_train, y_train_boost, final_xgb_model_test, y_test_boost)

In [49]:
print(f"Final model: Optimized XGBOOST with these columns: {final_xgb_model_input}; R2 for Train: {final_r2_train}, R2 for Test: {final_r2_test}")

Final model: Optimized XGBOOST with these columns: ['Adult Mortality', 'Income composition of resources', 'HIV/AIDS', 'Schooling']; R2 for Train: 0.9785060275440891, R2 for Test: 0.9511510355136336


# Stacking

In [50]:
reg1 = best_cb_model
reg2 = best_lgb_model

meta_reg = xgboost_def

In [51]:
# Define the stacking classifier
stacking_regressor = StackingCVRegressor(
    regressors=[reg1, reg2],
    meta_regressor=meta_reg,
    cv=5,
    use_features_in_secondary=True,
    verbose=1,
    random_state=42
    )

In [52]:
final_xgb_model_train.fillna(0, inplace=True)
final_xgb_model_test.fillna(0, inplace=True)

C:\Users\Shahbaz\AppData\Local\Temp\ipykernel_4300\1675250119.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_xgb_model_train.fillna(0, inplace=True)
C:\Users\Shahbaz\AppData\Local\Temp\ipykernel_4300\1675250119.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_xgb_model_test.fillna(0, inplace=True)


In [53]:
stacking_r2_train, stacking_r2_test = train_test_and_evaluate_model('Stacking Regressor with XGB, Optimized Catboost and Optimized LGBM', stacking_regressor, final_xgb_model_train, y_train_boost, final_xgb_model_test, y_test_boost)

0:	learn: 8.9984360	total: 2.29ms	remaining: 1.95s
1:	learn: 8.4690924	total: 4.25ms	remaining: 1.81s
2:	learn: 7.9643957	total: 6.84ms	remaining: 1.94s
3:	learn: 7.5031136	total: 8.75ms	remaining: 1.86s
4:	learn: 7.0891909	total: 10.6ms	remaining: 1.81s
5:	learn: 6.7060543	total: 12.5ms	remaining: 1.76s
6:	learn: 6.3474962	total: 14.5ms	remaining: 1.76s
7:	learn: 6.0251512	total: 18.6ms	remaining: 1.97s
8:	learn: 5.7201293	total: 20.6ms	remaining: 1.94s
9:	learn: 5.4299908	total: 22.6ms	remaining: 1.91s
10:	learn: 5.1792053	total: 24.5ms	remaining: 1.88s
11:	learn: 4.9392291	total: 26.3ms	remaining: 1.85s
12:	learn: 4.7137522	total: 28.1ms	remaining: 1.82s
13:	learn: 4.4974411	total: 31.5ms	remaining: 1.89s
14:	learn: 4.3012986	total: 33.5ms	remaining: 1.87s
15:	learn: 4.1272736	total: 35.2ms	remaining: 1.85s
16:	learn: 3.9628217	total: 37ms	remaining: 1.82s
17:	learn: 3.8192631	total: 38.7ms	remaining: 1.8s
18:	learn: 3.6793511	total: 40.5ms	remaining: 1.78s
19:	learn: 3.5505165	tota

In [54]:
print(f"Stacking model: Optimized XGBOOST with these columns: {final_xgb_model_input}; R2 for Train: {stacking_r2_train}, R2 for Test: {stacking_r2_test}")

Stacking model: Optimized XGBOOST with these columns: ['Adult Mortality', 'Income composition of resources', 'HIV/AIDS', 'Schooling']; R2 for Train: 0.9816239558582408, R2 for Test: 0.9523744141051013
